In [2]:
pwd

'/home/nakyung/projects/BDAIFin/notebooks'

In [4]:
import os
os.chdir("..")

In [8]:
import OFFLINE.models.type_package as tp
import importlib
importlib.reload(tp)


<module 'OFFLINE.models.type_package' from '/home/nakyung/projects/BDAIFin/OFFLINE/models/type_package.py'>

In [9]:
import importlib
import pandas as pd
import numpy as np

# 1) 데이터 로드
df = pd.read_parquet("data/online/train_oss.parquet")
df_f = df[df["fraud"] == 1].copy()

# 2) client_state 적용 (너 코드 그대로 사용)
from OFFLINE.features.client_state import add_client_state_feature

class Cfg:
    client_col = "client_id"

df_f_feat = add_client_state_feature(df_f, Cfg)

# 3) type_package 실행
import OFFLINE.models.type_package as tp
importlib.reload(tp)

cfg = tp.TypeDiscoveryConfig(
    label_col="fraud",
    id_cols=("client_id","card_id","merchant_id"),
    feature_cols=None,
    use_scaler=True,
    fillna_value=0.0,

    # (선택) client_top_mccg_prev one-hot 하고 싶으면 True
    onehot_client_top_mccg_prev=False,

    # GMM
    gmm_k_min=2,
    gmm_k_max=12,
    gmm_covariance_type="full",
    compute_silhouette=True,
    silhouette_sample_size=30000,

    # HDBSCAN
    hdb_min_cluster_size=30,
    hdb_min_samples=None
)

X, feat_cols, scaler = tp.make_type_discovery_matrix(df_f_feat, cfg)
print("X:", X.shape, "n_feat:", len(feat_cols))

gmm, best_k, scores = tp.fit_gmm_best(X, cfg)
print("best_k:", best_k)
display(scores)

labels = gmm.predict(X)
prof = tp.profile_clusters(df_f_feat, labels, feat_cols, topn=8)
print("cluster_counts(top):", sorted(prof["cluster_counts"].items(), key=lambda x:-x[1])[:5])

# HDBSCAN 
if tp._HDBSCAN_AVAILABLE:
    hdb, hlabels = tp.fit_hdbscan(X, cfg)
    hprof = tp.profile_clusters(df_f_feat, hlabels, feat_cols, topn=8)
    print("HDB n_clusters:", hprof["n_clusters_ex_noise"], "noise:", hprof["noise_ratio"])


X: (8570, 72) n_feat: 72
best_k: 12


,k,bic,aic,silhouette
0,2,-6.872953e+05,-7.254048e+05,0.327600
1,3,-8.626839e+05,-9.198518e+05,0.050057
2,4,-9.122196e+05,-9.884459e+05,0.064966
3,5,-1.051003e+06,-1.146288e+06,0.068170
4,6,-9.401807e+05,-1.054524e+06,0.040475
5,7,-1.185466e+06,-1.318867e+06,0.035556
6,8,-1.226528e+06,-1.378988e+06,0.045741
7,9,-1.248641e+06,-1.420159e+06,0.041249
8,10,-1.213658e+06,-1.404234e+06,0.037715
9,11,-1.270859e+06,-1.480494e+06,0.041137


cluster_counts(top): [(8, 3171), (5, 1307), (3, 839), (1, 838), (2, 694)]


/home/nakyung/.local/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/nakyung/.local/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


HDB n_clusters: 6 noise: 0.05682613768961493


In [10]:
hdb, hlabels = tp.fit_hdbscan(X, cfg)
hprof = tp.profile_clusters(df_f_feat, hlabels, feat_cols, topn=10)

for cid in sorted([c for c in hprof["cluster_counts"].keys() if c != -1]):
    print("\ncluster", cid, "n=", hprof["cluster_counts"][cid])
    print("top:", hprof["cluster_top_features_absdiff"][cid])
print("\nnoise ratio:", hprof["noise_ratio"])


/home/nakyung/.local/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/nakyung/.local/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(



cluster 0 n= 41
top: ['total_debt', 'yearly_income', 'per_capita_income', 'credit_limit', 'amount', 'amount_mean', 'months_to_expire', 'credit_score', 'amount_diff', 'tx_year']

cluster 1 n= 53
top: ['total_debt', 'credit_limit', 'yearly_income', 'per_capita_income', 'amount_mean', 'amount_diff', 'amount', 'credit_score', 'months_to_expire', 'birth_year']

cluster 2 n= 59
top: ['total_debt', 'yearly_income', 'per_capita_income', 'credit_limit', 'amount', 'amount_mean', 'amount_diff', 'current_age', 'birth_year', 'err_bad_card_number']

cluster 3 n= 87
top: ['total_debt', 'per_capita_income', 'yearly_income', 'credit_limit', 'amount_mean', 'amount', 'credit_score', 'months_to_expire', 'amount_diff', 'birth_year']

cluster 4 n= 7767
top: ['total_debt', 'yearly_income', 'per_capita_income', 'credit_limit', 'amount_mean', 'amount_diff', 'amount', 'months_to_expire', 'credit_score', 'client_mccg_Food_Daily_cnt_prev']

cluster 5 n= 76
top: ['credit_limit', 'yearly_income', 'per_capita_incom